# Working on Data Shape

In many programming languages, the programmer has to declare the dimensions of an array statically, and it is often possible to operate on the individual items using programmer-written loops.

In contrast to this, because APL processes arrays in their entirety, it is important to be able to manage the dimensions of an array dynamically.
This is why this chapter presents a number of new tools that will help you perform these tasks.

We have already studied functions which create arrays with specific shapes:

| Name | Syntax | Explanation | Section |
| :- | :-: | :- | :- |
| Reshape | `x⍴y` | Creates a new array with dimensions `x` and the data from `y`. | [see here](./Data-and-Variables.ipynb#Create-a-List-or-a-Matrix) |
| Catenate | `x,y` | Creates a new array by gluing `x` and `y` together. | [see here](./Some-Primitive-Functions.ipynb#Concatenate) |
| Ravel | `,y` | Creates a vector from any array. | [see here](./Some-Primitive-Functions.ipynb#Ravel) |
| Compress | `x/y` | Selects parts of an array. | [see here](./Some-Primitive-Functions.ipynb#Basic-Approach:-Compress) |
| Replicate | `x/y` | Generally replicates the items of an array. | [see here](./Some-Primitive-Functions.ipynb#General-Case:-Replicate) |
| Indexing | `x[y]` | Creates a new array, often with modified dimensions. | [see here](./Data-and-Variables.ipynb#Indexing) |
| Index function | `x⌷y` | Creates a new array, often with modified dimensions. | [see here](./Data-and-Variables.ipynb#The-Index-Function) |

## Take and Drop

### Take and Drop Applied to Vectors

#### Starter

There are two functions, _take_ (`↑`) and _drop_ (`↓`), that can be used to extract or remove a number of elements from the beginning or end of a vector.
The number is the left argument and its sign controls if the extraction/removal is done from the beginning or end of the vector.

 - _Take_ extracts the vector's head or tail; and
 - _Drop_ removes the vector's head or tail, and hence selects the remaining part.

Let us test these functions:

In [1]:
nums ← 56 66 19 37 44 20 18 23 68 70 82

With a positive left argument `n`, _take_ extracts the first `n` items:

In [2]:
4↑nums

56 66 19 37

It works on any kind of data (numbers, text),

In [3]:
5↑'My name is Bond'

My na

including nested arrays:

In [4]:
2↑(6 2)(35 33 26 21)(42 73)(1 2 3)

┌───┬───────────┐
│6 2│35 33 26 21│
└───┴───────────┘

With a negative left argument `n`, it extracts the last `|n` items of the vector, in their normal order:

In [5]:
¯3↑nums

68 70 82

In [6]:
¯6↑'Mississippi'

ssippi

Sometimes, these two operations are referred to as taking the "head" of the vector (with a positive left argument) and taking the "tail" of the vector (with a negative left argument).

With a positive left argument `n`, _drop_ removes the first `n` items and returns the tail:

In [7]:
4↓nums

44 20 18 23 68 70 82

In [8]:
5↓'My name is Bond'

me is Bond

With a negative left argument `n`, it removes the last `|n` items and returns the head:

In [9]:
¯6↓'Mississippi'

Missi

<!--begin remark-->
***Remark***:

 > You will have noticed that `4↑nums` and `¯7↓nums` both gave the same result `56 66 19 37`.
 >
 > At first sight it would appear that there is no need for both of these functions, and that one or other of _take_ and _drop_ is redundant.
 > There are, however, some differences that make it necessary to have both functions, as we will soon see.
<!--end-->

#### Be Careful

Do not confuse the two following expressions.

We can take the **last** three items of a vector:

In [10]:
¯3↑nums

68 70 82

Or we can take the first three items, and then change their sign:

In [11]:
-3↑nums

¯56 ¯66 ¯19

On another note, the result of _take_ and _drop_ applied to a vector remains a vector, even if it has only one item:

In [12]:
⍴1↑nums

1

In [13]:
⍴10↓nums

1

Although the results of the two expressions above would contain only one item, they are 1-item vectors, and not scalars.

#### Produce Empty Vectors

Of course, if you take no items, or if you drop all the items, the result is an empty vector of the same type (numeric or character) as the original vector:

In [14]:
0↑nums

In [15]:
22↓nums

In [16]:
14↓'Empty'

Although all of the results above are empty vectors, we know that they are different if the original vectors have different types of data:

In [17]:
(22↓nums)≡(14↓'Empty')

0

Take a look at [this section](./Data-and-Variables.ipynb#Empty-Arrays) for a reminder of how empty arrays work.

#### Take More Cash Than You Have

The _take_ function has a very special property: it allows you to take more items than there really are.
If so, it pads the result with _fill items_; zeroes for a numeric vector, and blanks for a text vector:

In [18]:
cash ← 45 23 18 92

If we take too many items, zeroes are appended to the vector:

In [19]:
7↑cash

45 23 18 92 0 0 0

With negative left arguments, starting from the tail, the zeroes are placed before the existing items:

In [20]:
¯9↑cash

0 0 0 0 0 45 23 18 92

For character vectors, spaces are appended:

In [21]:
12↑'Invisible'

Invisible

However, you have to be careful because those do not show:

In [22]:
(12↑'Invisible'),'.'

Invisible .

Starting from the right, with a negative left argument, the blanks are appended to the left and are a bit easier to see:

In [23]:
¯12↑'Visible'

Visible

In fact, the concept of _fill item_ is a bit more complex than this; it will be studied in detail in [a later section](./Nested-Arrays-(Continued).ipynb#Prototype-Fill-Item).

The concept of "taking more than you have" is sometimes referred to as _overtaking_.
This is an application of _take_ that cannot be performed using _drop_ alone.

This property applies equally to empty vectors; they are filled with as many zeroes or blanks as specified.
This means that the result will be different for empty numeric and empty character vectors:

In [24]:
]display 4↑⍬

┌→──────┐
│0 0 0 0│
└~──────┘

In [25]:
]display 4↑''

┌→───┐
│ │
└────┘

### Three Basic Applications

#### Determine the Type of a Variable

The property we have just seen can be used to determine whether an array is numeric or character, provided that it is simple and homogeneous (neither _mixed_ nor _nested_).
The method is simple: create an empty vector "filled" with the array (`0⍴⍵`), then take one item of it (`1↑`), and compare with `0`.
This will return `1` (true) for a numeric array, and `0` (false) for a character array.
A little dfn will do that for us:

In [26]:
TypeOf ← {0=1↑0⍴⍵}

In [27]:
TypeOf nums

1

In [28]:
TypeOf 'Character vector'

0

This function wouldn't work on a mixed or nested variable.
Again, you will understand why when we talk about _fill items_ in [this section](./Nested-Arrays-(Continued).ipynb#Prototype-Fill-Item).
We shall see in [a later chapter](./System-Interfaces.ipynb) that APL has a _system function_ that does the job much better.

#### Change a Vector into a Matrix

Sometimes you want a variable `var` to be a matrix, although you are not sure of its current rank.

If it is already a matrix, you want to leave it unchanged, and if it is a vector, you want to change it into a one-row matrix.
The following function should help:

In [29]:
HorMat ← {(¯2↑1,⍴⍵)⍴⍵}

Notice that the shape of the following matrix remains unchanged:

In [30]:
⍴HorMat 3 5⍴⍳5

3 5

But the vector is "promoted" into a matrix:

In [31]:
⍴HorMat 'This is a vector'

1 16

Let's break down how `HorMat` works:

 1. we first append `1` to the shape of the argument, which gives `1 3 5` for the matrix and `1 16` for the vector;
 2. next, we keep only the last two items, which gives `3 5` for the matrix and `1 16` for the vector;
 3. the final _reshape_ returns an unchanged matrix, or transforms a vector intro a matrix.

We can change vectors into 1-column matrices with a very similar function, which also leaves matrices unchanged:

In [32]:
VerMat ← {(2↑(⍴⍵),1)⍴⍵}

In [33]:
VerMat 'Vertically?'

V
e
r
t
i
c
a
l
l
y
?

#### Calculate Growth Rates

Let us imagine a business with a turnover which has grown over 12 years.
The variable `tome` is TurnOver in Millions of Euros:

In [34]:
tome ← 56 59 67 64 60 61 68 73 78 75 81 84

We want to calculate the difference between each year and the next; how do we achieve this?

In [35]:
1↓tome

59 67 64 60 61 68 73 78 75 81 84

In [36]:
¯1↓tome

56 59 67 64 60 61 68 73 78 75 81

All that remains is to subtract the results of these expressions one from the other, item by item:

In [37]:
(1↓tome)-(¯1↓tome)

3 8 ¯3 ¯4 1 7 5 5 ¯3 6 3

If instead of subtraction we used division, we would calculate (with some adjustments) the rates of growth instead of the differences.
Let us upt that in a small dfn, and apply it:

In [38]:
Growth ← {100×((1↓⍵)÷(¯1↓⍵))-1}

In [39]:
2⍕Growth tome

5.36 13.56 ¯4.48 ¯6.25 1.67 11.48 7.35 6.85 ¯3.85 8.00 3.70

### Take and Drop Applied to Arrays

The functions _take_ and _drop_ can be applied to any array so long as the left argument contains a number of items that is equal to or smaller than the number of dimensions (the _rank_) of the array.

In the expressions `ns↑array` and `ns↓array`, `≢ns` must be less than or equal to `≢⍴array`.

For the examples that follow, we shall work on the following matrix:

In [40]:
mat ← 3 4⍴13 52 33 81 42 62 70 47 51 73 28 19

`mat` is an integer matrix but _take_ and _drop_ work the same for character arrays, and even for _nested_ and/or _mixed_ arrays.

#### Maximal-Length Left Argument

We start by exploring what happens when the left argument has the maximal length it can have, that is, when `(≢ns)=≢⍴array` in the expression `ns↑array` or `ns↓array`.

If we want to take the 2 top rows and the 3 left columns of `mat`, we write

In [41]:
2 3↑mat

13 52 33
42 62 70

The elements of the left argument are paired up with the axes of the right argument array, and that is why `2 3↑mat` takes 2 _rows_ and then 3 _columns_, because the first axis specifies the number of rows and the second one the number of columns.

In order to drop the top row and 2 columns _from the right_, we write

In [42]:
1 ¯2↓mat

42 62
51 73

If we take 5 rows starting from the bottom and 3 columns starting from the left, we get

In [43]:
¯5 3↑mat

0 0 0
 0 0 0
13 52 33
42 62 70
51 73 28

The two extra rows are added on the top of `mat` because we asked for 5 rows _starting from the bottom_.

Dropping 1 row starting from the bottom and 3 columns starting from the left, we get

In [44]:
¯1 3↓mat

81
47

In the example above, we have dropped 3 out of the 4 columns available, so there is only one left, but it is still a 1-column matrix; it has not been changed into a vector.
This is similar to how `1↑vector` in a vector returns a 1-element vector, not a scalar.
In short, the functions _take_ and _drop_ return a result with the same rank as the right argument, regardless of the number of elements in the result.

As for vectors, it is often possible to use _take_ or _drop_ interchangeably to obtain the same result:

In [45]:
2 ¯3↑mat

52 33 81
62 70 47

In [46]:
¯1 1↓mat

52 33 81
62 70 47

#### Short Left Argument

<!-- (TODO) Write about Leading Axis Theory -->
If one or more of the trailing dimensions of the array are to remain unchanged, then one need only specify the parameters for the leading dimensions.

For example, suppose that we want to extract the first 2 rows of `mat`.
We know its shape is `3 4`, so we can easily write

In [47]:
2 4↑mat

13 52 33 81
42 62 70 47

But if we didn't know its shape in advance, we would have to resort to something like

In [48]:
(2,¯1↑⍴mat)↑mat

13 52 33 81
42 62 70 47

which is a bit cumbersome and, as it turns out, far from optimal.

In a matrix, the first axis denotes the rows, so we can pick the first 2 rows by just specifying the `2` in the left argument to _take_:

In [49]:
2↑mat

13 52 33 81
42 62 70 47

This works for arrays of arbitrarily high rank.
For example, recall the 3D array `prod`, that contained monthly (12) production values of 2 assembly lines, throughout the course of 5 years:

In [50]:
⎕RL ← 73
⎕← prod ← ?5 2 12⍴50

9 16 42 50 2 3 11 45 17 37 29 36
34 19 32 12 37 15 46 24 49 28 36 9
 
29 5 45 23 27 4 23 16 39 22 22 2
46 20 47 17 18 25 13 38 42 38 45 28
 
37 19 5 10 30 49 16 18 46 47 47 3
39 23 41 20 6 40 21 22 40 49 20 1
 
13 36 12 40 12 15 24 20 21 12 19 25
 8 48 22 37 33 1 39 30 50 50 8 35
 
38 31 24 40 23 7 20 34 35 6 19 27
41 1 14 50 17 42 6 35 12 48 30 29

We can get the production values for the first 2 years with:

In [51]:
2↑prod

9 16 42 50 2 3 11 45 17 37 29 36
34 19 32 12 37 15 46 24 49 28 36 9
 
29 5 45 23 27 4 23 16 39 22 22 2
46 20 47 17 18 25 13 38 42 38 45 28

But we can also get the production values for the first 4 years and only for the first assembly line:

In [52]:
4 1↑prod

9 16 42 50 2 3 11 45 17 37 29 36
 
29 5 45 23 27 4 23 16 39 22 22 2
 
37 19 5 10 30 49 16 18 46 47 47 3
 
13 36 12 40 12 15 24 20 21 12 19 25

Naturally, _drop_ can also be used with a left argument whose length is smaller than the rank of the argument.
For example, we can drop the first row of a matrix:

In [53]:
1↓mat

42 62 70 47
51 73 28 19

Or we can drop the production values of the _last_ year and of the first assembly line:

In [54]:
¯1 1↓prod

34 19 32 12 37 15 46 24 49 28 36 9
 
46 20 47 17 18 25 13 38 42 38 45 28
 
39 23 41 20 6 40 21 22 40 49 20 1
 
 8 48 22 37 33 1 39 30 50 50 8 35

In [the exercises](#Exercises) you can find a rule to compute the shape of the final result of a _take_ (or _drop_) operation in terms of the left argument and the shape of the right argument.

#### Short Left Argument with Axis Specification

When you provide a left argument that has less elements than the number of axes on the right argument, the left argument defaults to referring to the leading axes of the right argument.
However, we can use _take_ and _drop_ with _axis_ `[]` to specify other axes.

For example, the first 2 rows of `mat` were obtained with

In [55]:
2↑mat

13 52 33 81
42 62 70 47

Using _axis_, we can just as easily get the first 2 columns of `mat`:

In [56]:
2↑[2]mat

13 52
42 62
51 73

This also extends to higher rank.
If we want the information regarding the first 3 years, and only the final 6 months of the year (but for all assembly lines), we can write:

In [57]:
3 ¯6↑[1 3]prod

11 45 17 37 29 36
46 24 49 28 36 9
 
23 16 39 22 22 2
13 38 42 38 45 28
 
16 18 46 47 47 3
21 22 40 49 20 1

## Laminate

We have previously used _catenate_ to glue one array to another; let us now look at a new method.

We shall work with the following two character matrices:

In [58]:
⎕← names ← 3 4⍴'AlexAnnaMark'

Alex
Anna
Mark

In [59]:
⎕← surnames ← 3 4⍴'GainNairTyte'

Gain
Nair
Tyte

The _catenation_ of those two matrices will not change the _rank_ of the arrays, and thus give another matrix, as we saw in [a previous chapter](./Some-Primitive-Functions.ipynb#Concatenate):

In [60]:
names,surnames

AlexGain
AnnaNair
MarkTyte

In [61]:
names⍪surnames

Alex
Anna
Mark
Gain
Nair
Tyte

If both matrices have exactly the same shape, it is possible to join them together along a new dimension to make a 3D array.
Because this operation produces a result of higher rank than its arguments, it is called _laminate_ rather than _catenate_.

The symbol representing _catenate_ and _laminate_ is the same (`,`), but when the comma is used as _laminate_ it is always used with a fractional _axis_.

The two arrays we intend to _laminate_ have the same shape: `3 4`.
Because we are going to laminate 2 arrays, the new dimension will have a length of 2, and the shape of the result will be some combination of `3`, `4`, and `2`.
Let us examine all the possibilities:

 - <code>2 3   4&nbsp;&nbsp;</code> – the new dimension is inserted before the first dimension;
 - <code>&nbsp;&nbsp;3 2 4&nbsp;&nbsp;</code> – the new dimension is inserted between the first and second dimensions;
 - <code>&nbsp;&nbsp;3   4 2</code> – the new dimension is inserted after the second dimension.

To obtain these three different results, we shall use _laminate_ with a fractional axis to specify where the new dimension is to be inserted:

 - `names,[0.5]surnames` will produce a result of shape <code>2 3   4&nbsp;&nbsp;</code>;
 - `names,[1.5]surnames` will produce a result of shape <code>&nbsp;&nbsp;3 2 4&nbsp;&nbsp;</code>; and
 - `names,[2.5]surnames` will produce a result of shape <code>&nbsp;&nbsp;3   4 2</code>.

Beware that we could have equally used `⍪`.

Here are the three cases:

In [62]:
names,[0.5]surnames

Alex
Anna
Mark
 
Gain
Nair
Tyte

In [63]:
names,[1.5]surnames

Alex
Gain
 
Anna
Nair
 
Mark
Tyte

In [64]:
names,[2.5]surnames

AG
la
ei
xn
 
AN
na
ni
ar
 
MT
ay
rt
ke

In fact, the value of the axis specifier just identifies the _position_ of the new dimension relative to the values 1 and 2, so it could be any other fractional value between 0 and 1, or 1 and 2, or 2 and 3, respectively.

Hence, the three matches that we see below:

In [65]:
(names,[0.295]surnames)≡names,[0.5]surnames

1

In [66]:
(names,[1.643]surnames)≡names,[1.5]surnames

1

In [67]:
(names,[2.107]surnames)≡names,[2.5]surnames

1

Of course, it would be somewhat obtuse to use such axis specifications, and programmers conventionally use "`n.5`" values, like the ones in our examples.

### Applications to Vectors and Scalars

Now that we understand the reason for the fractional axis, which is perhaps initially somewhat surprising, we can apply _laminate_ to all kind of arrays.

#### Laminate Applied to Vectors

Let us use both character and numeric vectors:

In [68]:
t1 ← 'tomatoes'
t2 ← 'potatoes'
n1 ← 14 62 32 88
n2 ← 10×⍳4

If we catenate them, we still obtain vectors:

In [69]:
t1,t2

tomatoespotatoes

In [70]:
n1,n2

14 62 32 88 10 20 30 40

But if we instead laminate them, we obtain matrices with either 2 rows or 2 columns:

In [71]:
t1,[0.5]t2

tomatoes
potatoes

In [72]:
t1,[1.5]t2

tp
oo
mt
aa
tt
oo
ee
ss

In [73]:
n1,[0.5]n2

14 62 32 88
10 20 30 40

In [74]:
n1,[1.5]n2

14 10
62 20
32 30
88 40

Of course, since we are working with 1-dimensional arrays, we cannot specify an axis equal to or greater than 2.

There is also no reason for why you wouldn't be able to laminate a character vector with a numeric vector, for example:

In [75]:
t1,[0.5]n1,n2

t o m a t o e s
14 62 32 88 10 20 30 40

#### Laminate Scalars with Vectors

Scalars can be laminated with any array: they are repeated as many times as necessary to match the length of the new dimension:

In [76]:
n1,[0.5]0

14 62 32 88
 0 0 0 0

In [77]:
1,[1.5]n1

1 14
1 62
1 32
1 88

This can be used, for example, to underline a title:

In [78]:
title ← 'Laminate is good for you'

Without _laminate_, we must create a matrix with 2 rows, and as many columns as the length of `title`, filled with `title` itself, followed by as many dashes as the length of `title`: boring!

In [79]:
(2,≢title)⍴title,(≢title)⍴'-'

Laminate is good for you
------------------------

Now, with _laminate_, we just have to laminate a single dash; it will be repeated as many times as necessary:

In [80]:
title,[0.5]'-'

Laminate is good for you
------------------------

### Applications

#### Interlace Matrices

Do you remember that, in [the chapter about user-defined functions](./User-Defined-Functions.ipynb), we wrote a function to interlace two matrices?
It is no longer relevant; we can solve the problem more simply using _laminate_.

Can you see how? Give it some thought, if you will.

Take a look at the result of `names,[2.5]surnames` above.
You will see that the names are on the left and the surnames are on the right.
If we reshape that result with appropriate dimensions, we shall obtain `names` and `surnames` interlaced 😊 :

In [81]:
(1 2×⍴names)⍴names,[2.5]surnames

AGlaeixn
ANnaniar
MTayrtke

The result isn't easy to read, but the expression works!

Hence, our new implementation of `Interlace` could be

In [82]:
Interlace ← {(1 2×⍴⍺)⍴⍺,[2.5]⍵}

We can apply the same technique to matrices of forecasts and actuals, provided we define them first:

In [83]:
⎕RL ← 73
forecast ← 10×?4 6⍴55
⎕RL ← 73
actual ← forecast + ¯10+?4 6⍴20

In [84]:
forecast Interlace actual

90 89 160 166 420 420 500 508 20 12 30 23
110 111 450 453 170 177 370 365 290 284 360 352
340 349 190 192 320 329 120 115 510 515 370 374
150 160 460 467 240 234 520 519 490 485 280 283

#### Show Vectors

Sometimes you will have two vectors that are related, and you would like to display them side by side.
You might have a character vector and you would like to display it side by side with a Boolean vector indicating the positions of vowels in the character vector, so that you can verify visually if the result is correct:

In [85]:
text ← 'National Aeronautics and Space Administration'
vowels ← 0 1 0 1 1 0 1 0 0 0 1 0 1 0 1 1 0 1 0 0 0 1 0 0 0 0 0 1 0 1 0 0 0 0 1 0 1 0 0 0 1 0 1 1 0

In [86]:
text,[0.5]vowels

N a t i o n a l A e r o n a u t i c s a n d S p a c e A d m i n i s t r a t i o n
0 1 0 1 1 0 1 0 0 0 1 0 1 0 1 1 0 1 0 0 0 1 0 0 0 0 0 1 0 1 0 0 0 0 1 0 1 0 0 0 1 0 1 1 0

By looking at the above, you would be able to see more easily that your result missed the capital vowels, and you would then proceed to fixing your function.

This was a simple example of where laminating two vectors might prove useful, but this ad-hoc technique might also be useful in slightly more complex scenarios.
Suppose that we have four vectors containing information about certain products: their price, their current discount, their quantity in stock, and their availability (physical store, online, or both):

In [87]:
⎕← price ← 2 4 15 8 23
⎕← discount ← 0 0 25 25 35
⎕← stock ← 103 98 50 23 64
⎕← availability ← 'SSBOB'

2 4 15 8 23

0 0 25 25 35

103 98 50 23 64

SSBOB

The output shown above is not ideal, because each individual vector is displayed using its own natural format, and it is extremely difficult to connect the four related pieces of information to a specific product.
We can have these values displayed much better if we create a matrix.

To produce a matrix we will need to laminate two of the vectors and catenate the others (in rows or in columns).
The results are much easier to read:

In [88]:
price⍪discount⍪stock⍪[0.5]availability

2 4 15 8 23
 0 0 25 25 35
103 98 50 23 64
 S S B O B

In [89]:
price,discount,stock,[1.5]availability

2 0 103 S
 4 0 98 S
15 25 50 B
 8 25 23 O
23 35 64 B

In these examples, there is only one _laminate_, followed by as many _catenates_ as needed.

## Expand

### Basic Use

You remember that simple _compress_ uses a Boolean vector of 1s and 0s as a mask to include or exclude specific items of an array.

Simple _expand_ (specified by the `\` symbol) also uses a Boolean vector of 1s and 0s, but the 0s _insert_ new items into the array.
It is used as follows: `r ← pattern\argument`.

In this form, the Boolean vector left argument contains a 1 for each item of the right argument, and a 0 for each item to insert.
For example:

In [90]:
1 1 0 1 0 0 1 1 1\11 28 32 40 57 69

11 28 0 32 0 0 40 57 69

In [91]:
1 1 0 1 0 0 1 1 1\'Africa'

Af r ica

If the right argument is numeric, _expand_ inserts zeroes, and if it is a character vector, _expand_ inserts blanks as _fill items_.
For mixed or nested arrays, the concept of _fill item_ is more complex and, as mentioned before, will be explained in [a later section](./Nested-Arrays-(Continued).ipynb#Prototype-Fill-Item).

### Extended Definition

We can extend the behaviour of _extend_ to handle cases where the left argument is not a simple Boolean vector, but contains integers other than just 0s and 1s:

 - the amount of positive numbers in the left argument should match the length of the right argument;
 - for each positive item in the left argument, the corresponding item in the right argument is replicated as many times as is specified by that value;
 - each negative item in the left argument inserts an equivalent number of _fill items_ in the same position; and
 - zeroes in the left argument mean the same as `¯1`, and they each insert one fill item.

In the description above, we assumed that the right argument is a vector.
We will cover the fully generic case in a bit.

Because it is an extension of it, this new definition is fully compatible with the Boolean case we described before.

Here is an example:

In [92]:
1 1 0 3 ¯2 1 1 1\11 28 32 40 57 69

11 28 0 32 32 32 0 0 40 57 69

The first two items remain unchanged.
Then a zero inserts a zero in the result.
The next value is repeated 3 times, and the value `¯2` inserts 2 zeroes.
The last 3 items are unchanged.

The same thing can be done using a character vector:

In [93]:
1 1 0 3 ¯2 1 1 1\'expand'

ex ppp and

Because `0` and `¯1` have the same effect when used in the left argument of _expand_, we can obtain the same result with a different pattern:

In [94]:
1 1 ¯1 3 ¯2 1 1 1\'expand'

ex ppp and

Naturally, the function can work on any shape of array, in which case `\` acts on the last axis of the right argument, by default.

For example, if we take our `chemistry` matrix:

In [95]:
⎕← chemistry ← 3 5⍴'H2SO4CaCO3Fe2O3'

H2SO4
CaCO3
Fe2O3

We can insert one extra column by using _expand_ with a Boolean left argument:

In [96]:
1 1 0 1 1 1\chemistry

H2 SO4
Ca CO3
Fe 2O3

We can further specify the axis along which we want to act, to change the default:

In [97]:
1 0 1 0 1\[1]chemistry

H2SO4
 
CaCO3
 
Fe2O3

The axis specified can be the same as the default, making it redundant, but otherwise completely correct:

In [98]:
1 ¯3 1 1 1 0 3\[2]chemistry

H 2SO 444
C aCO 333
F e2O 333

With arrays of rank higher than 1, the description provided above remains the same, except that _extend_ no longer operates on single items of the right argument, but on its sub-arrays along the given axis (the last one, if left unspecified) instead.

_Expand_ can also be used on scalars; they are repeated as many times as necessary to fit the number of positive values in the left argument:

In [99]:
0 0 1 1 0 0 1 1\'A'

AA AA

The above is equivalent to

In [100]:
0 0 1 1 0 0 1 1\'AAAA'

AA AA

In [101]:
0 1 3 ¯2 1 1\71

0 71 71 71 71 0 0 71 71

### Expand Along First Axis

Like it was mentioned before, _expand_ works on the last dimension of an array.
To work on the first dimension, one can use the function `⍀` (<kbd>APL</kbd>+<kbd>.</kbd>):

In [102]:
1 1 ¯2 1⍀chemistry

H2SO4
CaCO3
 
 
Fe2O3

If one places an axis indication after the symbol `\` or `⍀`, the operation is processed according to the _axis_ operator, whichever of the two symbols is used.
For example:

 - `vec⍀[3]prod` and `vec\[3]prod` would be equivalent to `vec\prod`; and
 - `vec\[1]forecast` and `vec⍀[1]forecast` would be equivalent to `vec⍀forecast`.

## Reverse and Transpose

APL is also well endowed with functions which pivot data about an axis, and the axis is suggested by the shape of the symbol used.
The functions apply to both numeric and character data.
In the examples we are going to use a character matrix called `towns`:

In [103]:
⎕← towns ← 6 10⍴'Canberra  Paris     WashingtonMoscow    Martigues Mexico    '

Canberra 
Paris 
Washington
Moscow 
Martigues 
Mexico

The symbols `⌽` (<kbd>APL</kbd>+<kbd>Shift</kbd>+<kbd>5</kbd>) and `⊖` (<kbd>APL</kbd>+<kbd>Shift</kbd>+<kbd>7</kbd>) are used for two variants of the same function, which is called _reverse_.

The function `⍉` is called _transpose_ and its symbol is typed with <kbd>APL</kbd>+<kbd>Shift</kbd>+<kbd>6</kbd>.

Here are their effects:

In [104]:
(⊢towns) (⌽towns) (⊖towns) (⍉towns)

┌──────────┬──────────┬──────────┬──────┐
│Canberra │ arrebnaC│Mexico │CPWMMM│
│Paris │ siraP│Martigues │aaaoae│
│Washington│notgnihsaW│Moscow │nrssrx│
│Moscow │ wocsoM│Washington│bihcti│
│Martigues │ seugitraM│Paris │esioic│
│Mexico │ ocixeM│Canberra │r nwgo│
│ │ │ │r g u │
│ │ │ │a t e │
│ │ │ │ o s │
│ │ │ │ n │
└──────────┴──────────┴──────────┴──────┘

The symbols used (`⌽ ⊖ ⍉`) are self-describing, no effort is required to remember any of them because the position of the bar visually indicates which kind of transformation they stand for.

If you insert an axis specification after the symbols `⌽` or `⊖`, the operation is processed according to the _axis_ operator, whichever of the two symbols is used.
So, for example:

 - `⌽[1]matrix` and `⊖[1]matrix` are both equivalent to `⊖matrix`; and
 - `⌽[2]matrix` and `⊖[2]matrix` are both equivalent to `⌽matrix`.

### Caveats to be Aware Of

 - _Transpose_ has no effect on a vector, because it has only one axis:

In [105]:
⍉'I shall not move'

I shall not move

On a similar note,

 - The distracted APLer might expect `⊖` to do nothing on vectors, but that is _not_ what happens.
 
Visually, `⊖` hints at the fact that it "flips" its argument upside down, and a vector can't be flipped upside down because it is one-dimensional.
However, `⊖` is called _reverse **first**_, which means it always reverses the argument along its **first** dimension, even if that dimension is the _only_ dimension of the argument:

In [106]:
⊖'I shall not move...!?'

?!...evom ton llahs I

This confusion might arise from the visual similarity of the character vector above and

In [107]:
1 21⍴'I shall not move...!?'

I shall not move...!?

The latter is a 1-row matrix, which is why it _is_ left unchanged by `⊖`:

In [108]:
⊖1 21⍴'I shall not move...!?'

I shall not move...!?

 - _Transpose_ cannot be modified by an axis specifier, because it always operates on all of the dimensions of its argument.

 - _Transpose_ can be applied to arrays of any rank; let us try it with a 3D character array:

In [109]:
⎕← big ← names,[0.5]surnames

Alex
Anna
Mark
 
Gain
Nair
Tyte

In [110]:
⍉big

AG
AN
MT
 
la
na
ay
 
ei
ni
rt
 
xn
ar
ke

In [111]:
⍴big

2 3 4

In [112]:
⍴⍉big

4 3 2

You can see that `⍴⍉big` is equal to `⌽⍴big`, and the explanation of why this is the case will follow shortly.

All three primitives we just discussed also have (related) dyadic meanings, and those are introduced in two following sections.

## Rotate

The symbols `⌽` and `⊖`, when used dyadically, shift the items of the right argument in a circular manner.

The dyadic functions are called _rotate_.

### Rotate Vectors

Much like in the monadic case, the dyadic usage of `⌽` and `⊖` is identical when applied to vectors (we shall use `⌽` in our examples because of the visual clue that the symbol `⌽` gives with respect to the operation it performs).

Dyadic `⌽` (and `⊖`) expects an integer as the left argument, when the right argument is a vector, like so: `r ← n⌽vector`.

 - when `n` is positive, the first `n` items of `vector` are moved to the end. In other words, the vector is rotated to the left; and
 - when `n` is negative, the last `n` items of `vector` are moved to the beginning. In other words, the vector is rotated to the right:

In [113]:
7⌽'What did they do to my song?'

d they do to my song?What di

In the example above, the first 7 items of the vector (`'What di'`) have been moved to the back, whereas if we use a negative argument:

In [114]:
¯7⌽'What did they do to my song?'

y song?What did they do to m

the last 7 items of the vector (`'y song?'`) are moved to the front of the vector.

_Rotate_ can of course be applied to numeric vectors as well:

In [115]:
nums

56 66 19 37 44 20 18 23 68 70 82

In [116]:
3⌽nums

37 44 20 18 23 68 70 82 56 66 19

Do not confuse the following two expressions:

In [117]:
¯3⌽nums

68 70 82 56 66 19 37 44 20 18 23

In [118]:
-3⌽nums

¯37 ¯44 ¯20 ¯18 ¯23 ¯68 ¯70 ¯82 ¯56 ¯66 ¯19

The first one moves 3 items to the beginning, whilst the second expression moves the first 3 items to the end, and then changes the sign of the result (we saw something very similar when we talked about _take_).
It is all about being careful with the normal and high minus symbols!

### Rotate Higher-Rank Arrays

When applied to a matrix or higher-order array, `⌽` works on the last dimension, while `⊖` works on the first dimension.
This default behaviour can be overriden by an _axis_ specification.
To obtain a rotation along any other dimension, the _axis_ specification is mandatory.

_Rotate_ can be applied to any array, but we shall only demonstrate its application to matrices.

#### Uniform Rotation

In its simplest form, _rotate_ applies the same rotation to all the rows or columns of a matrix; let us see the result produced on a character matrix:

In [119]:
⎕← monMat ← 6 8⍴'January FebruaryMarch   April   May     June    '

January 
February
March 
April 
May 
June

In [120]:
2⌽monMat

nuary Ja
bruaryFe
rch Ma
ril Ap
y Ma
ne Ju

In [121]:
2⊖monMat

March 
April 
May 
June 
January 
February

In [122]:
¯2⊖monMat

May 
June 
January 
February
March 
April

#### Multiple Rotations

It is possible to apply a different rotation to each of the rows or to each of the columns.

In this case, the rotation is no longer indicated by a single value, but by a vector which specifies the amount by which each row or column will be moved.

In [123]:
chemistry

H2SO4
CaCO3
Fe2O3

In [124]:
¯1 0 2⌽chemistry

4H2SO
CaCO3
2O3Fe

Notice how `¯1` was used to rotate the first row, `0` was used to rotate the second row, and `2` was used to rotate the third row.

Using `⊖`, we can rotate columns:

In [125]:
monMat

January 
February
March 
April 
May 
June

In [126]:
1 0 2 ¯2 0 0 2 2⊖monMat

Far ar 
Mereua 
Aayuh 
Mpnrl 
Janc y 
Jubi ry

#### Application

_Rotate_ can provide very simple solutions to many tasks.
For example, let us count how many blanks appear at the end of each row of `monMat`:

In [127]:
+/' '=monMat

1 0 3 3 5 4

We can then use these values to move the blanks to the beginning of each row, thereby right-aligning the matrix:

In [128]:
(-+/' '=monMat)⌽monMat

January
February
 March
 April
 May
 June

## Dyadic Transpose

_Dyadic transpose_ is interesting only for arrays of rank higher than 2.
It rotates an array as if to show it from different angles.

Remember our variable called `prod`:

In [129]:
prod

9 16 42 50 2 3 11 45 17 37 29 36
34 19 32 12 37 15 46 24 49 28 36 9
 
29 5 45 23 27 4 23 16 39 22 22 2
46 20 47 17 18 25 13 38 42 38 45 28
 
37 19 5 10 30 49 16 18 46 47 47 3
39 23 41 20 6 40 21 22 40 49 20 1
 
13 36 12 40 12 15 24 20 21 12 19 25
 8 48 22 37 33 1 39 30 50 50 8 35
 
38 31 24 40 23 7 20 34 35 6 19 27
41 1 14 50 17 42 6 35 12 48 30 29

It is an array with 3 dimensions, which are respectively 5 years, 2 assembly lines, and 12 months.

Suppose we now want to reorganise it into an array of 2 assembly lines, 5 years, and 12 months: a dyadic transposition can do that for us.

The left argument of _dyadic transpose_ specifies the position that you want each dimension to appear in the result.

The shape of `prod` was

In [130]:
⍴prod

5 2 12

but now we want it to become `2 5 12`.
The years should become the 2nd dimension, the assembly lines should become the 1st dimension, and the months should remain the 3rd dimension, hence the transposition vector will be `2 1 3`:

In [131]:
2 1 3⍉prod

9 16 42 50 2 3 11 45 17 37 29 36
29 5 45 23 27 4 23 16 39 22 22 2
37 19 5 10 30 49 16 18 46 47 47 3
13 36 12 40 12 15 24 20 21 12 19 25
38 31 24 40 23 7 20 34 35 6 19 27
 
34 19 32 12 37 15 46 24 49 28 36 9
46 20 47 17 18 25 13 38 42 38 45 28
39 23 41 20 6 40 21 22 40 49 20 1
 8 48 22 37 33 1 39 30 50 50 8 35
41 1 14 50 17 42 6 35 12 48 30 29

Alternatively, if we want to reorganise it into an array of 2 assembly lines, 12 months, and 5 years, the method will be the same:

The required new shape is `2 12 5`.

When all dimensions are different from each other, the transposition vector can be generated using dyadic _iota_:

In [132]:
⎕← tv ← 2 12 5 ⍳ ⍴prod

3 1 2

In [133]:
tv⍉prod

9 29 37 13 38
16 5 19 36 31
42 45 5 12 24
50 23 10 40 40
 2 27 30 12 23
 3 4 49 15 7
11 23 16 24 20
45 16 18 20 34
17 39 46 21 35
37 22 47 12 6
29 22 47 19 19
36 2 3 25 27
 
34 46 39 8 41
19 20 23 48 1
32 47 41 22 14
12 17 20 37 50
37 18 6 33 17
15 25 40 1 42
46 13 21 39 6
24 38 22 30 35
49 42 40 50 12
28 38 49 50 48
36 45 20 8 30
 9 28 1 35 29

!["Skilled APLer that is confident with their tools."](res/APLer-Skilled_Orange.png)

## Exercises

**Exercise 1**:

You are given this matrix:

In [134]:
⎕← xg1 ← 3 5⍴1 9 5 3 6 5 4 8 2 3 7 7 6 2 6

1 9 5 3 6
5 4 8 2 3
7 7 6 2 6

Try to produce each of the three following matrices, using first only _take_, and then only _drop_:

```APL
5 3 6
8 2 3
```

```APL
5 4 8 2
7 7 6 2
```

```APL
9 5 3
4 8 2
7 6 2
```

**Exercise 2**:

With `xg1` again, how could you produce this:

```APL
1 9 5 3 6 0
5 4 8 2 3 0
7 7 6 2 6 0
0 0 0 0 0 0
```

**Exercise 3**:

Write a dyadic function `TakeShape` that computes the _shape_ of the result of doing `ns↑array`.
The left argument to `TakeShape` will be `ns` and the right argument will be `⍴array`.
You can assume that the left argument and right arguments are valid and make sense together.

Also implement `DropShape`, that does the same thing as `TakeShape`, but for _drop_ instead of for _take_.

In [135]:
TakeShape ← {}

In [136]:
2 ¯3 TakeShape 1 5 10
⍝ 2 3 10

In [137]:
DropShape ← {}

In [138]:
15 4 ¯3 DropShape 10 10 10
⍝ 0 6 7

**Exercise 4**:

Write two dyadic functions, `TakeAsDrop` and `DropAsTake`, that take an array on the right and a vector on the left.
`r ← v TakeAsDrop a` is such that `(r↓a)≡v↑a` whenever possible, and conversely for `DropAsTake`: `r ← v DropAsTake` is such that `(r↑a)≡v↓a` whenever possible.
Test your functions with your solutions to exercise 1.

**Exercise 5**:

Write a function which "highlights" all the vowels of a given character vector by placing an arrow under them:

In [139]:
ShowVowels ← {}

In [140]:
ShowVowels 'This function works properly'
⍝ This function works properly
⍝   ↑   ↑   ↑↑   ↑      ↑ ↑  ↑

**Exercise 6**:

Some matrices are mainly filled with zeroes, like the matrix `xg4` shown below:

In [141]:
xg6 ← 4 7⍴0
xg6[(1 3)(1 6)(2 2)(3 1)(3 3)(3 7)(4 5)] ← 8 3 7 6 2 1 4
xg6

0 0 8 0 0 3 0
0 7 0 0 0 0 0
6 0 2 0 0 0 1
0 0 0 0 4 0 0

These are called _sparse matrices_.

A large sparse matrix may occupy a lot of memory.
To reduce the memory consumption, we can ravel the matrix,

In [142]:
,xg6

0 0 8 0 0 3 0 0 7 0 0 0 0 0 6 0 2 0 0 0 1 0 0 0 0 4 0 0

retaining only the positive values together with their position in the ravel:

In [143]:
positiveValues ← 8 3 7  6  2  1  4
positions      ← 3 6 9 15 17 21 26

If now we add the shape of the matrix on the left, we have all the necessary information to restore the original matrix when required:

```APL
 4 8 3 7  6  2  1  4
 7 3 6 9 15 17 21 26
⍝↑ shape of the original matrix
```

Can you write

 - a function which creates this compact form (let us call the function `Contraction`); and
 - a function `Restore` which retrieves the original matrix from its compact form?

**Exercise 7**:

In a given character vector, we would like to replace all the occurrences of a given letter with blanks:

In [144]:
sentence ← 'Panama is a canal between Atlantic and Pacific'
Whiten ← {}

In [145]:
'a' Whiten sentence
⍝ P n m  is   c n l between Atl ntic  nd P cific

Find a solution using _expand_.

**Exercise 8**:

Write a dyadic function to centre a title above a character matrix, like this:

In [146]:
OnTop ← {}

In [147]:
'2007' OnTop monMat
⍝   2007
⍝ --------
⍝ January
⍝ February
⍝ March
⍝ April
⍝ May
⍝ June

**Exercise 9**:

You are given this matrix:

In [148]:
⎕← xg9 ← 3 9⍴'oeornlhtu n siduothf uogYti'

oeornlhtu
 n siduot
hf uogYti

What is the result of the expression `¯3 ¯1 3⌽(-2 1 0 1 0 2 1 2 0)⊖xg9`?

**Exercise 10**:

You are given a Boolean vector like the following:

In [149]:
⎕← xg10 ← 1 0 0 1 1 1 0 1 0 0 0 1 1 0 0 0 0 1 1 0 0 0 0 1 0 0

1 0 0 1 1 1 0 1 0 0 0 1 1 0 0 0 0 1 1 0 0 0 0 1 0 0

We would like to find a list of `n` contiguous zeroes in this vector.
Write a function which gives the position of the first zero of the first such list found.
If there is no list of `n` zeroes, the function is supposed to return 0.
Loops are (of course) strictly forbidden!

In [150]:
Free ← {}

In [151]:
3 Free xg10
⍝ 9

In [152]:
6 Free xg10
⍝ 0

In [153]:
4 Free xg10
⍝ 14

**Exercise 11**:

The following is a long matrix of names:

In [154]:
⎕← xg11 ← 11 8⍴'Emily   Luciano Paul    Oxana   Thor    Carmen  VeronicaWilliam VladimirMonica  Colette '

Emily 
Luciano 
Paul 
Oxana 
Thor 
Carmen 
Veronica
William 
Vladimir
Monica 
Colette

Write a function to split this matrix into slices, and position these slices one next to the other, like this:

In [155]:
Split ← {}

In [156]:
3 Split xg11
⍝ Emily    Thor     Vladimir
⍝ Luciano  Carmen   Monica
⍝ Paul     Veronica Colette
⍝ Oxana    William

The number of slices is passed as the left argument and a blank is inserted between the slices.

**Exercise 12**:

You are given a numeric vector:

In [157]:
xg12 ← 22 22 74 74 74 74 30 65 65 65 19

It has the same number of items as there are names in the variable `xg9` from the previous exercise.
It is composed of groups of identical codes.

Write a function which displays side by side this vector of codes and the associated matrix of names, with an empty line inserted each time the code changes, like this:

In [158]:
Expand ← {}

In [159]:
xg12 Expand xg11
⍝ 22  Emily
⍝ 22  Luciano
⍝
⍝ 74  Paul
⍝ 74  Oxana
⍝ 74  Thor
⍝ 74  Carmen
⍝
⍝ 30  Veronica
⍝
⍝ 65  William
⍝ 65  Vladimir
⍝ 65  Monica
⍝
⍝ 19  Colette

## The Specialist's Section

<br />
<center>
<i>Each chapter is followed by a "Specialist's Section" like this one. This section is dedicated to skilled APLers, who wish to improve their knowledge.</i>
You will find here rare or complex usages of the concepts presented in this chapter, or discover extended explanations which need the knowledge of some symbols that will be seen much further in the book.
    
<b>If you are exploring APL for the first time, skip this section and go to the next chapter.</b>
</center>

### More About Laminate

Here is a formal definition of the conditions required to laminate two variables `a` and `b`.

In the expression `r ← a,[axis]b`,

 - it is mandatory that (⍴a)≡(⍴b), unless one of them is a scalar;
 - `axis` must be a value between `⎕IO-1` and `⎕IO+≢⍴a`; and
 - the shape of `r` is given by the expression `((⌊axis↑⍴a),2,((⌈axis)↓⍴a)`.

Examine the second rule: it is obvious that `axis` can be negative if the _index origin_ is set to 0.
The axis can also be negative for _mix_ and _ravel_ with axis.

Let us use the following two vectors:

In [160]:
a ← 41 27 88 11
b ← 39 63 12 69

In order to produce the matrix

In [161]:
2 4⍴a,b

41 27 88 11
39 63 12 69

we write

In [162]:
a,[0.5]b

41 27 88 11
39 63 12 69

when `⎕IO ← 1`, but if we set

In [163]:
⎕IO ← 0

then the expression becomes

In [164]:
a,[¯0.5]b

41 27 88 11
39 63 12 69

In [165]:
⎕IO ← 1

### Dyadic Transpose

#### Conditions

We said that _dyadic transpose_ can be thought of as a way to observe an array from different positions.
For this usage there is a certain rule to follow.

To transpose an array with `a⍉b`, we must have `a[⍋a]≡⍳≢⍴b`.

In other words, `a` must be composed of all the values of `⍳≢⍴b` taken in any order, or, mathematically speaking, `a` must be a permutation of the axes of `b`.

#### Diagonal Sections of an Array

_Dyadic transpose_ can also be used to select the items from an array which have two or more identical coordinates.
Such selections are called "diagonal sections" of the array.

For example, let us use the following array:

In [166]:
⎕← y ← 3 3 4⍴⍳36

1 2 3 4
 5 6 7 8
 9 10 11 12
 
13 14 15 16
17 18 19 20
21 22 23 24
 
25 26 27 28
29 30 31 32
33 34 35 36

Look at the result of the following expression:

In [167]:
1 1 2⍉y

1 2 3 4
17 18 19 20
33 34 35 36

We have specified that both the first and second dimensions are to become the first dimension of the result.
This conflict is resolved by extracting the items on the diagonal between the dimensions which are merged.

The items on this diagonal have identical first and second coordinates, much like the identical nature of the first and second values in the left argument `1 1 2⍉y`.
The result is a section of the "cube".

The expression `2 2 1⍉y` would give the same result, but transposed:

In [168]:
2 2 1⍉y

1 17 33
2 18 34
3 19 35
4 20 36

In [169]:
(1 1 2⍉y)≡⍉2 2 1⍉y

1

<!--begin rules style=remark-->
***Rules***:

 > This special use of `a⍉b` must also follow some rules:
 >
 >  - `(≢a)≡(≢⍴b)`
 >  - `∧/a∊⍳≢⍴b`
 >  - `∧/(⍳⌈/a)∊a`
<!--end-->

The first two rules are not unique to this special usage of _dyadic transpose_, and the third rule means that the items of `a` must be consecutive integers, starting from `⎕IO`.

So, for the array `y` shown above, the only possible sections are:

In [170]:
(1 1 1)(1 1 2)(1 2 1)(1 2 2)(2 1 1)(2 1 2)(2 2 1)(⊣,[.5]⍉¨)⊂y

┌───────┬───────────┬────────┬────────┬────────┬───────┬───────┐
│1 1 1 │1 1 2 │1 2 1 │1 2 2 │2 1 1 │2 1 2 │2 2 1 │
├───────┼───────────┼────────┼────────┼────────┼───────┼───────┤
│1 18 35│ 1 2 3 4│ 1 5 9│ 1 6 11│ 1 13 25│1 14 27│1 17 33│
│ │17 18 19 20│14 18 22│13 18 23│ 6 18 30│5 18 31│2 18 34│
│ │33 34 35 36│27 31 35│25 30 35│11 23 35│9 22 35│3 19 35│
│ │ │ │ │ │ │4 20 36│
└───────┴───────────┴────────┴────────┴────────┴───────┴───────┘

If the required conditions are satisfied, the selection can be thought of as having two steps:

 1. the left argument `a` is examined to see which of the coordinates are identical.

For example, a left argument equal to `1 2 1` or `2 1 2` will select items which have their first and their coordinates identical.
A left argument equal to `1 1 2` or `2 2 1` will select items which have their first two coordinates identical.

`1 2 2⍉y` and `2 1 1⍉y` would select the items which have their last two coordinates equal, that is,

```APL
 1  6 11
13 18 23
25 30 35
```

So, now we know which items from `y` we must work with in the next step.

 2. the selected values are repositioned using a normal dyadic transposition, the left argument of which is composed of the unique values of `a` obtained by `∪a`.

For example, we said that both `1 2 2⍉y` and `2 1 1⍉y` would select the same set of values, shown above.
This little matrix will then be transposed using a left argument equal to `1 2` (the matrix remains unchanged) or `2 1` (the matrix is transposed):

In [171]:
1 2 2⍉y

1 6 11
13 18 23
25 30 35

In [172]:
2 1 1⍉y

1 13 25
 6 18 30
11 23 35

We recommend that you try to work through the other possibilities to make sure you understand them.

#### Diagonal Section of a Matrix

In the case of a matrix, the only possible diagonal section is specified by `1 1`.
It selects what is called the main diagonal of the matrix.

For example, here is how you would select the values along the main diagonal of `forecast`:

In [173]:
1 1⍉forecast

90 450 320 520

In [174]:
forecast

90 160 420 500 20 30
110 450 170 370 290 360
340 190 320 120 510 370
150 460 240 520 490 280

Notice that for a matrix `m`, we have `(≢1 1⍉m)=⌊/⍴m`.
This means that the diagonal section of a rectangular matrix has as many elements as the shorter side of the matrix, like in the `forecast` example above.

## Solutions

The following solutions we propose are not necessarily the “best” ones; perhaps you will find other solutions that we have never considered.
APL is a very rich language, and due to the general nature of its primitive functions and operators there are always plenty of different ways to express different solutions to a given problem.
Which one is “the best” depends on many things, for example the level of experience of the programmer, the importance of system performance, the required behaviour in border cases, the requirement to meet certain programming standards and also personal preferences.
This is one of the reasons why APL is so pleasant to teach and to learn!

We advise you to try and solve the exercises before reading the solutions!

**Exercise 1**:

Starting from `xg1`,

In [175]:
⎕← xg1 ← 3 5⍴1 9 5 3 6 5 4 8 2 3 7 7 6 2 6

1 9 5 3 6
5 4 8 2 3
7 7 6 2 6

we want to extract sub-matrices using _take_ and _drop_.
Because we are dealing with a matrix – a rank 2 array – the left argument must be a 2-element vector.
The first element determines how many rows we take (or drop) and from where (top or bottom), while the second element determines how many columns we take (or drop) and from where (left or right).

Let us go through the three matrices, one by one:

 - ```APL
5 3 6
8 2 3
```

This matrix has elements from the _first_ two rows and from the _last_ three columns of `xg1`, so we can build it with

In [176]:
2 ¯3↑xg1

5 3 6
8 2 3

Conversely, we can build that matrix by ignoring the _last_ row and the _first_ two columns:

In [177]:
¯1 2↓xg1

5 3 6
8 2 3

 - ```APL
 5 4 8 2
7 7 6 2
```

This matrix has elements from the _last_ two rows and from the _first_ four columns, so we build it with

In [178]:
¯2 4↑xg1

5 4 8 2
7 7 6 2

Conversely, we can build it by ignoring the _first_ row and the _last_ column of `xg1`:

In [179]:
1 ¯1↓xg1

5 4 8 2
7 7 6 2

 - ```APL
9 5 3
4 8 2
7 6 2
```

Finally, this matrix contains all three rows but the 3 columns that make it up are from the _middle_ of `xg1`, so there is no single number that we can use to build this matrix in a single _take_.
Instead, what we can do is two successive _takes_: first we take the _first_ 4 columns and then the _last_ 3 columns of that, or first we take the _last_ 4 columns and then the _first_ 3 of that:

In [180]:
3 3↑3 ¯4↑xg1

9 5 3
4 8 2
7 6 2

We do a similar thing with _drop_:

In [181]:
0 1↓0 ¯1↓xg1

9 5 3
4 8 2
7 6 2

**Exercise 2**:

We want to produce the matrix

```APL
1 9 5 3 6 0
5 4 8 2 3 0
7 7 6 2 6 0
0 0 0 0 0 0
```

from `xg1`.
We immediately notice that `xg1` is in the upper-left corner of this larger matrix, and we see there are an extra row and column of zeroes, which are certainly the result of overtaking:

In [182]:
4 6↑xg1

1 9 5 3 6 0
5 4 8 2 3 0
7 7 6 2 6 0
0 0 0 0 0 0

**Exercise 3**:

Let us start with `TakeShape`, that is supposed to compute the _shape_ of the result of doing `ns↑array`.

`r ← ns TakeShape ⍴array` should be such that `r≡⍴ns↑array`, and notice that the right argument to `TakeShape` is `⍴array`, not the array itself.

From reading the specification of _take_ and from the previous exercises, we can start to see that the absolute value of the left argument indicates the length of each dimension.
However, the solution isn't just

In [183]:
TakeShape ← {|⍺}

Why not?
Because the left argument to _take_ may be shorter than the number of dimensions of the array we are working with, in which case the trailing dimensions are left untouched.
To fix our function, we just need to check if there are trailing dimensions that are left untouched and, in case there are any, we should tack them to the right of `|ns`:

In [184]:
TakeShape ← {(|⍺),((≢⍺)-≢⍵)↑⍵}

In [185]:
2 ¯3 TakeShape 1 5 10
⍝ 2 3 10

2 3 10

How does it work?

 - `≢⍵` is the number of dimensions of the array and `≢⍺` is the number of dimensions of the left argument to _take_, so `(≢⍵)-≢⍺` is the number of trailing dimensions that are left untouched;
 - we then use _take_ to reach the lengths of those dimensions, so they show up in the final result. However, we want to _take_ from the end, so we would need `-(≢⍵)-≢⍺`; instead, we flip the subtraction in the first place to get `(≢⍺)-≢⍵`; and
 - we tack those trailing dimensions to the absolute value of the left argument.

The problem statement specifies that the arguments to `TakeShape` will always be relative to a usage of _take_ that makes sense, so we also know that `(≢⍺)-≢⍵` is always 0 or negative.

We can follow a similar train of thought with `DropShape`, except that now `⍺` doesn't tell us the length of each dimension, but rather the number of elements that are dropped along that dimension.
For `DropShape`, we can take advantage of overtaking to make sure that the left argument has the same length as the right argument.
Finally, we just need to make sure we don't try to drop too many elements along any dimension, because the final result cannot have dimensions with negative length:

In [186]:
DropShape ← {0⌈⍵-|(≢⍵)↑⍺}

In [187]:
15 4 ¯3 DropShape 10 10 10
⍝ 0 6 7

0 6 7

**Exercise 4**:

This exercise wants us to convert left arguments of _take_ into left arguments of _drop_ and vice-versa, and doing so means we need to understand the lengths of the resulting dimensions, but that is what we have been doing in the previous exercises.

Start by imagining how it works for a vector `v` of length `l`.
 - if we write `t↑v`, for positive `t`, the corresponding left argument for _drop_ would be `-l-t`; but
 - if we write `(-t)↑v`, then the corresponding left argument for _drop_ would be `l-t`.

Now we just need to find a way of unifying the two cases:

In [188]:
TakeAsDrop ← {(-×⍺)×⍵-|⍺}

The expression `⍵-|⍺` determines the magnitude of the corresponding _drop_ left argument and the expression `(-×⍺)` corrects the sign.

This is a great starting point, but our implementation has three issues that we need to solve.
One of them should become apparent here:

In [189]:
10 TakeAsDrop 5

5

If we try to _take_ 10 elements from a vector of length 5, we end up with a (larger) vector of length 10, so there is no left argument to _drop_ that would have the same effect.
Arguably, the most sensible thing to do here would be to return `0`, because that is the left argument for a _drop_ that would have the closest result possible to the original _take_.
To make this change, what we need to do is make sure that the magnitude of the argument to _drop_ is never below zero:

In [190]:
TakeAsDrop ← {(-×⍺)×0⌈⍵-|⍺}
10 TakeAsDrop 5

0

The second issue, that really is a bug, has to do with what happens if we _take_ 0 elements:

In [191]:
0 TakeAsDrop 5

0

If we take 0 elements from a vector of length 5, that would be the same as dropping 5 elements from that vector.
However, we say it is the same as dropping 0 elements from that vector, which is wrong.
The problem is that `(-×⍺)` evaluates to `0` in this case, whipping out the correct magnitude computed by `0⌈⍵-|⍺`.
One way to fix that is by nudging the values of `⍺` slightly, so that their signs remain the same for all integers, except for 0:

In [192]:
TakeAsDrop ← {(-×⍺-0.5)×0⌈⍵-|⍺}

In [193]:
0 TakeAsDrop 5

5

The third and final issue, is that `TakeAsDrop` doesn't handle cases where the left argument is shorter than the right argument:

In [194]:
3 4 TakeAsDrop 10 8 6

LENGTH ERROR: Mismatched left and right argument shapes
TakeAsDrop[0] TakeAsDrop←{(-×⍺-0.5)×0⌈⍵-|⍺}
                                       ∧


The simplest way to fix this is by shortening `⍵` to match the length of `⍺`.
This works, because the trailing dimensions that are not covered by the left argument to _take_ can also be omitted in the left argument to _drop_:

In [195]:
TakeAsDrop ← {(-×⍺-0.5)×0⌈((≢⍺)↑⍵)-|⍺}
3 4 TakeAsDrop 10 8 6

¯7 ¯4

As for the `DropAsTake` function... It is exactly the same as the `TakeAsDrop` one!

**Exercise 5**:

It is clear that an integral part of this exercise will be finding the vowels in the given character vector, and we can use the _membership_ primitive for that:

In [196]:
'This function works properly'∊'aeiouyAEIOUY'

0 0 1 0 0 0 1 0 0 0 1 1 0 0 0 1 0 0 0 0 0 0 1 0 1 0 0 1

Next, we want to turn that Boolean mask into a character vector with arrows `↑` where the ones are.
If we take advantage of the fact that the blank space `' '` is the fill element for character vectors, we can use the _expand_ primitive that we just learned about:

In [197]:
('This function works properly'∊'aeiouyAEIOUY')\'↑'

↑ ↑ ↑↑ ↑ ↑ ↑ ↑

To wrap it up, all we need to do is laminate this character vector with the original vector:

In [198]:
ShowVowels ← {⍵,[.5](⍵∊'aeiouyAEIOUY')\'↑'}
ShowVowels 'This function works properly'

This function works properly
 ↑ ↑ ↑↑ ↑ ↑ ↑ ↑

**Exercise 6**:

Let's work with the following matrix:

In [199]:
xg6 ← 4 7⍴0
xg6[(1 3)(1 6)(2 2)(3 1)(3 3)(3 7)(4 5)] ← 8 3 7 6 2 1 4
xg6

0 0 8 0 0 3 0
0 7 0 0 0 0 0
6 0 2 0 0 0 1
0 0 0 0 4 0 0

In order to contract the matrix, we start by raveling it:

In [200]:
,xg6

0 0 8 0 0 3 0 0 7 0 0 0 0 0 6 0 2 0 0 0 1 0 0 0 0 4 0 0

And we also list all the indices of all the positions in the matrix:

In [201]:
⍳≢,xg6

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28

What we now want is to keep only the indices of non-zero values.
If we start by laminating the numbers and the positions together, we can then use a Boolean mask to figure out what things to keep:

In [202]:
xg6r ← ,xg6
(xg6r≠0)/xg6r,[.5]⍳≢xg6r

8 3 7 6 2 1 4
3 6 9 15 17 21 26

The final step is catenating the original shape of the matrix on the left:

In [203]:
]dinput
Contraction ← {
    r ← ,⍵
    (⍴⍵),(r≠0)/r,[.5]⍳≢r
}

In [204]:
Contraction xg6

4 8 3 7 6 2 1 4
7 3 6 9 15 17 21 26

In order to implement the function `Restore`, we can create an empty ravel of the matrix which we then populate with the correct values.
After populating, we just need to reshape the ravel into the original matrix by making use of the shape information that is on the first column of the input:

In [205]:
]dinput
Restore ← {
    shape ← ⍵[;1]
    ravel ← (×/shape)⍴0
    ravel[1 1↓⍵] ← ¯1 1↓⍵
    shape⍴ravel
}

In [206]:
Restore Contraction xg6

0 0 8 0 0 3 0
0 7 0 0 0 0 0
6 0 2 0 0 0 1
0 0 0 0 4 0 0

**Exercise 7**:

When the problem statement mentions blanks and _expand_, it hints at the fact that the blank space is the fill element for character vectors, and we know that _expand_ inserts fill elements when it finds zeroes.

A first hunch might be that we just need to create a Boolean mask that has a 1 for each character we want to keep, and then we use _expand_ with that, but unfortunately that does not work:

In [207]:
sentence ← 'Panama is a canal between Atlantic and Pacific'
(sentence≠'a')\sentence

LENGTH ERROR
      (sentence≠'a')\sentence
                    ∧


The problem is that the left argument to _expand_ needs to have as many positive elements as there are elements in the right argument.
In order to fix this, what we can do is remove the letter we want to replace with blanks and only _then_ use _expand_.
We can remove the occurrences of the letter with a _compress_ or with the _without_ primitive:

In [208]:
Whiten ← {(⍵≠⍺)\⍵~⍺}
'a' Whiten sentence

P n m is c n l between Atl ntic nd P cific

In [209]:
]dinput
Whiten ← {
    m ← ⍵≠⍺
    m\m/⍵
}

In [210]:
'a' Whiten sentence

P n m is c n l between Atl ntic nd P cific

**Exercise 8**:

If we want to centre something, that means we will need to rotate it to the correct place.
When we receive the title, we first pad it with enough blank spaces so that it can be catenated on top of the matrix:

In [211]:
padded ← (1↓⍴monMat)↑'2007'

In [212]:
≢padded

8

We overtake to achieve this effect.
The next thing is rotating some of the trailing spaces to the front, so that the title becomes centred.
To do that, we need to figure out how much larger the matrix is in relation to the title, and then we divide that by 2 and round, in case there are an odd number of spaces:

In [213]:
rot ← ⌈((1↓⍴monMat)-≢'2007')÷2

Now we can rotate the padded title by _minus_ that amount (otherwise we would be rotating in the wrong direction):

In [214]:
(-rot)⌽padded

2007

Now it is just a matter of catenating this to the top of the matrix, together with the `-` underline.
Let us put everything together in a dfn:

In [215]:
]dinput
OnTop ← {
    c ← 1↓⍴⍵
    rot ← ⌈(c-≢⍺)÷2
    padded ← c↑⍺
    ((-rot)⌽padded)⍪'-'⍪⍵
}

In [216]:
'2007' OnTop monMat

2007 
--------
January 
February
March 
April 
May 
June

Notice that rounding up or down changes the position of the extra space when there are an odd number of spaces around the title:

In [217]:
'107' OnTop monMat

107 
--------
January 
February
March 
April 
May 
June

**Exercise 9**:

Best way to check if your solution is correct is by actually running the code:

In [218]:
⎕← xg9 ← 3 9⍴'oeornlhtu n siduothf uogYti'

oeornlhtu
 n siduot
hf uogYti

In [219]:
¯3 ¯1 3⌽(-2 1 0 1 0 2 1 2 0)⊖xg9

You found
the right
solution

**Exercise 10**:

Let us prototype our solution with the vector that follows, and let us assume we want to find a run of 4 zeroes.

In [220]:
⎕← xg10 ← 1 0 0 1 1 1 0 1 0 0 0 1 1 0 0 0 0 1 1 0 0 0 0 1 0 0

1 0 0 1 1 1 0 1 0 0 0 1 1 0 0 0 0 1 1 0 0 0 0 1 0 0

We can start by asking _where_ in our vector the 1s are:

In [221]:
⍸xg10

1 4 5 6 8 12 13 18 19 24

This is useful because we know that the 0s are _between_ the 1s.

However, if our vector starts and/or ends with 0s, those will not be enclosed by 1s and so we need to prepend and append a 1 to be sure all runs of 0s are surrounded by 1s:

In [222]:
⎕← ones ← ⍸1,xg10,1

1 2 5 6 7 9 13 14 19 20 25 28

The next thing we do is figure out the distances between the consecutive 1s, so that we know the lengths of the runs of 0s:

In [223]:
⎕← runLengths ← ¯1+(1↓ones)-(¯1↓ones)

0 2 0 0 1 3 0 4 0 4 2

Now we just use the lengths of the runs to compress the initial `ones` vector.
However, we can't do it directly:

In [224]:
(4=runLengths)/ones

LENGTH ERROR
      (4=runLengths)/ones
                    ∧


The problem is that the vector `ones` contains the beginning and end of each run of zeroes, whereas the vector `runLengths` only has the corresponding length.
In short, `ones` contains one element too many, that corresponds to the 1 that was catenated to the end of the argument vector.
So, we need to drop the last element of `ones` before compressing:

In [225]:
(4=runLengths)/¯1↓ones

14 20

Finally, we take the first element of that:

In [226]:
1↑(4=runLengths)/¯1↓ones

14

At this point, we can figure out the approximate location of our runs of zeroes, we just need to make sure that we return the exact index required by the problem statement.
The problem statement wanted us to return the index of the first 0 of such a run.
We worked with indices of 1s, and not 0s, but we also catenated a 1 at the beginning of the vector, thus shifting all the indices of the 1s.
This means that, even though we are working with indices of 1s, the final result matches the corresponding index of the first 0.

Let's wrap our code in a dfn:

In [227]:
]dinput
Free ← {
    ones ← ⍸1,⍵,1
    runLengths ← ¯1+(1↓ones)-(¯1↓ones)
    1↑(⍺=runLengths)/¯1↓ones
}

What if the right argument does not have a run with the required length?
Does our function return 0 like the problem statement requires?

In [228]:
6 Free xg10

0

It does...
But how?

When we filter for runs of the required length, we get nothing, because there are no such runs:

In [229]:
(6=runLengths)/¯1↓ones

In other words, we get an empty numeric vector:

In [230]:
⍬≡(6=runLengths)/¯1↓ones

1

If our vector is _empty_, then trying to use _take_ to get its first element corresponds to over taking, because we are trying to retrieve too many elements.
When that is the case, we know that the function _take_ will put as many _fill elements_ as needed, and so it returns a single 0 as a result of doing "one take" on an empty vector:

In [231]:
1↑⍬

0

Here is an alternative solution.
First, we stack the vector on top of itself as many times as required by the desired run length (let us say it is `n ← 4`):

In [232]:
n ← 4
⎕← mat ← n (≢xg10)⍴xg10

1 0 0 1 1 1 0 1 0 0 0 1 1 0 0 0 0 1 1 0 0 0 0 1 0 0
1 0 0 1 1 1 0 1 0 0 0 1 1 0 0 0 0 1 1 0 0 0 0 1 0 0
1 0 0 1 1 1 0 1 0 0 0 1 1 0 0 0 0 1 1 0 0 0 0 1 0 0
1 0 0 1 1 1 0 1 0 0 0 1 1 0 0 0 0 1 1 0 0 0 0 1 0 0

Next, we use _rotate_ `⌽` to rotate each row one more element than the previous one, so that each column represents `n` consecutive elements of the vector:

In [233]:
⎕← rot ← (¯1+⍳n)⌽mat

1 0 0 1 1 1 0 1 0 0 0 1 1 0 0 0 0 1 1 0 0 0 0 1 0 0
0 0 1 1 1 0 1 0 0 0 1 1 0 0 0 0 1 1 0 0 0 0 1 0 0 1
0 1 1 1 0 1 0 0 0 1 1 0 0 0 0 1 1 0 0 0 0 1 0 0 1 0
1 1 1 0 1 0 0 0 1 1 0 0 0 0 1 1 0 0 0 0 1 0 0 1 0 0

In other words, if you look at the top row, you will notice that the 3 elements below a number match the 3 elements to its right.
For example, the 4th element in the 1st row is

In [234]:
rot[1;4]

1

The 3 elements to its right are

In [235]:
rot[1;5 6 7]

1 1 0

and the three elements below it are

In [236]:
rot[2 3 4;4]

1 1 0

Now, the runs that we care about correspond to columns that are completely 0.
Using a reduction, we can figure out which columns have 1s and which do not:

In [237]:
∨⌿rot

1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1

To finish up, we can do something similar to what we did before:

In [238]:
1↑⍸~∨⌿rot

14

This is what this solution would look like in a dfn:

In [239]:
]dinput
Free2 ← {
    mat ← ⍺ (≢⍵)⍴⍵
    rot ← (¯1+⍳⍺)⌽mat
    1↑⍸~∨⌿rot
}

In [240]:
3 Free2 xg10

9

In [241]:
6 Free2 xg10

0

In [242]:
4 Free2 xg10

14

**Exercise 11**:

We start with our matrix of names:

In [243]:
⎕← xg11 ← 11 8⍴'Emily   Luciano Paul    Oxana   Thor    Carmen  VeronicaWilliam VladimirMonica  Colette '

Emily 
Luciano 
Paul 
Oxana 
Thor 
Carmen 
Veronica
William 
Vladimir
Monica 
Colette

Let us also assume we want to make `n ← 3` slices:

In [244]:
n ← 3

If we look at the problem statement, we see that the names must be laid out top to bottom, and only then left to right.
However, in APL, the _ravel order_ of arrays is first left to right, and _then_ top to bottom.
So it seems like we have to fight APL's natural order in some way...
The key to doing that is understanding that we actually have 3 dimensions along which we want to navigate:

 1. we want to walk along one dimension that holds a single name,
 2. then we want to go down along the column of names,
 3. and then we want to go right along the columns of names.

This hints at the fact that we should reshape our data to have 3 dimensions, where one dimension holds the slices, another holds all the names, and the last one holds the characters that compose a name.

The first thing that we need to do, in order to break up the matrix with the names, into an array with 3 dimensions, is figure out the length of each dimension.
`n ← 3` indicates that we want 3 slices, but how many names are there in each slice?

In [245]:
(height width) ← ⍴xg11
height÷n

3.66667

This doesn't divide evenly, so we use the _ceiling_ primitive to round up, and then we use _take_ to create empty names to pad the slice:

In [246]:
sliceSize ← ⌈height÷n
⎕← names ← (sliceSize×n)↑xg11

Emily 
Luciano 
Paul 
Oxana 
Thor 
Carmen 
Veronica
William 
Vladimir
Monica 
Colette

Maybe you didn't notice, but now there is an empty row added to the matrix.
So, the next step is reshaping the matrix `names` into a 3D array:

In [247]:
⎕← slices ← n sliceSize width ⍴ names

Emily 
Luciano 
Paul 
Oxana 
 
Thor 
Carmen 
Veronica
William 
 
Vladimir
Monica 
Colette

Now we have an array where the dimensions match the dimensions we want, but now we need to reorder the dimensions so that they do what we want when we reshape the 3D array back into a matrix.
We see that the 3rd dimension is already correct, because the names are laid out along the 3rd (last) dimension.
This means the only thing we can try to do is reorder the first and second dimensions:

In [248]:
2 1 3⍉slices

Emily 
Thor 
Vladimir
 
Luciano 
Carmen 
Monica 
 
Paul 
Veronica
Colette 
 
Oxana 
William

After the reordering of the two leading dimensions, we can now reshape the 3D array into a matrix:

In [249]:
sliceSize (n×width)⍴2 1 3⍉slices

Emily Thor Vladimir
Luciano Carmen Monica 
Paul VeronicaColette 
Oxana William

This is almost perfect, we just need to add blanks between all the slices.
In order to do so, we can catenate a blank space at the end of the array (making it one character wider) and then dropping the last column of superfluous blanks:

In [250]:
0 ¯1↓sliceSize (n×width+1)⍴2 1 3⍉slices,' '

Emily Thor Vladimir
Luciano Carmen Monica 
Paul Veronica Colette 
Oxana William

This can be written as a dfn:

In [251]:
]dinput
Split ← {
    (h w) ← ⍴⍵
    nh ← ⌈h÷⍺
    names ← (nh×⍺)↑⍵
    slices ← ⍺ nh w⍴names
    0 ¯1↓nh (⍺×w+1)⍴2 1 3⍉slices,' '
}

In [252]:
3 Split xg11

Emily Thor Vladimir
Luciano Carmen Monica 
Paul Veronica Colette 
Oxana William

For reference, we provide another implementation that is _not_ correct, where we just do some reshaping.
In this alternative solution, the names get listed from left to right, and then from top to bottom.

In [253]:
]dinput
SplitWrong ← {
    (h w) ← ⍴⍵
    nh ← ⌈h÷⍺
    0 ¯1↓nh(n×w+1)⍴(n×nh×w+1)↑,xg11,' '
}

In [254]:
3 SplitWrong xg11

Emily Luciano Paul 
Oxana Thor Carmen 
Veronica William Vladimir
Monica Colette

**Exercise 12**:

For this exercise, we are given an extra vector `xg12`:

In [255]:
xg12 ← 22 22 74 74 74 74 30 65 65 65 19

It is fairly straightforward to put the labels next to the names:

In [256]:
⎕← labeled ← ⍕xg12,' ',xg11

22 Emily 
22 Luciano 
74 Paul 
74 Oxana 
74 Thor 
74 Carmen 
30 Veronica
65 William 
65 Vladimir
65 Monica 
19 Colette

The final step is creating the blank lines that separate the different labels.

For that, we need to create a Boolean vector that we can give to _expand_.
This is what we need to create:

In [257]:
1 1 0 1 1 1 1 0 1 0 1 1 1 0 1⍀labeled

22 Emily 
22 Luciano 
 
74 Paul 
74 Oxana 
74 Thor 
74 Carmen 
 
30 Veronica
 
65 William 
65 Vladimir
65 Monica 
 
19 Colette

In order to create that vector, we will want to figure out the indices of the above vector that are 0.
To do so, we start by finding where the labels change:

In [264]:
⎕← indices ← ⍸(¯1↓xg12)≠1↓xg12

2 6 7 10

However, if we look at the Boolean vector above, these are the indices of the 0s:

In [263]:
⍸~1 1 0 1 1 1 1 0 1 0 1 1 1 0 1

3 8 10 14

Can you see the pattern?

Here is the mapping that we need to do:

 - 2 goes to 3
 - 6 goes to 8
 - 7 goes to 10
 - 10 goes to 14

In [265]:
indices+⍳≢indices

3 8 10 14

Every index that we found in `indices` points to the last position of a run of labels, and we wanted to insert a new 0 in the position after the run ends and before the next run begins.
However, if we imagine we are working from the left, every time we insert a new position between different labels, we shift part of the vector to the right.
That is why we need to add the _iota_ of the _tally_ of `indices`, to correct for those shifts.

Now we use these indices to create the Boolean vector we need:

In [268]:
⎕← exp ← ~(⍳(≢indices)+≢xg12)∊indices+⍳≢indices

1 1 0 1 1 1 1 0 1 0 1 1 1 0 1

In [270]:
exp⍀labeled

22 Emily 
22 Luciano 
 
74 Paul 
74 Oxana 
74 Thor 
74 Carmen 
 
30 Veronica
 
65 William 
65 Vladimir
65 Monica 
 
19 Colette

In [271]:
]dinput
Expand ← {
    labeled ← ⍕⍺,' ',⍵
    indices ← ⍸(¯1↓⍺)≠1↓⍺
    exp ← ~(⍳(≢indices)+≢xg12)∊indices+⍳≢indices
    exp⍀labeled
}

In [272]:
xg12 Expand xg11

22 Emily 
22 Luciano 
 
74 Paul 
74 Oxana 
74 Thor 
74 Carmen 
 
30 Veronica
 
65 William 
65 Vladimir
65 Monica 
 
19 Colette